In [4]:
import numpy as np
import cv2

# Params for ShiTomasi corner detection
feature_params = dict(maxCorners=100, qualityLevel=0.1, minDistance=3, blockSize=3)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

cap = cv2.VideoCapture(r'C:\Users\ffah\Desktop\Projects\BEng Project\SIIT ML\Robotics Lecture\Robotics_code\week5\Figures\slow.mp4')

# Read the first frame in the entire video sequence
ret, first_frame = cap.read()
old_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

# Finds the strongest corners in the first frame using Shi-Tomasi method
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Creates an image filled with zero intensities with the same dimensions as the frame for later drawing purposes
mask = np.zeros_like(first_frame)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculates sparse optical flow using the Lucas-Kanade method
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draws the optical flow tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()

        # Convert points to integers
        a, b, c, d = int(a), int(b), int(c), int(d)

        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv2.circle(frame, (a, b), 3, color[i].tolist(), -1)

    img = cv2.add(frame, mask)

    cv2.imshow("Sparse Optical Flow", img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27 or cv2.getWindowProperty("Sparse Optical Flow", cv2.WND_PROP_VISIBLE) < 1:
        break

    # Update the previous frame
    old_gray = frame_gray.copy()

    # Update previous good feature points
    p0 = good_new.reshape(-1, 1, 2)

# Release resources and close all windows
cv2.destroyAllWindows()
cap.release()